In [20]:
consumer_key = '3Q3uGWsEUkybJGk7fRQfkk4yx'
consumer_secret = 'X2yHUZtN9vCc4bfRLewg3BboldtO0IETWYQezYVfyJ3NMvyI03'
access_token = '1455894515026329604-dVdCUbzL9f3MIQ8yt0KzuK5cCxJagu'
access_secret = 'V3mIL3I9bG954uoB4iFYTArFw6NAztvsriOtfhpe9aYRv'

In [21]:
import json
import os

def read_db(filepath):
    if not os.path.exists(filepath):
        return {}
    with open(filepath, 'r') as f:
        try:
            return json.load(f)
        except json.JSONDecodeError:
            return {}

db_data = read_db(r'db\1.json')


In [ ]:
db_data['graphql']['user']['id']

In [23]:
import instaloader
L = instaloader.Instaloader()
user_id = db_data['graphql']['user']['id']
profile = instaloader.Profile.from_id(L.context, user_id)


In [ ]:
followers = profile.get_followees()

In [29]:
L.login(user='desmond_wongggg_',passwd='90280036a')

In [40]:
import json

with open('instagram_data.json', 'r') as file:
    data = json.load(file)
    
with open('instagram_data.json','w') as file:
    json.dump(data,file,indent=4)

In [50]:
import networkx as nx
import matplotlib.pyplot as plt

graph = nx.read_edgelist(r'twitter\384685422.edges',create_using=nx.DiGraph(), nodetype=int)
#graph = nx.read_edgelist(r'twitter_combined.txt',create_using=nx.DiGraph(), nodetype=int)

In [51]:
print("#Nodes:", graph.number_of_nodes())
print("#Edges:", graph.number_of_edges()) 

#Nodes: 174
#Edges: 6563


In [54]:
pos = nx.kamada_kawai_layout(graph)
nx.draw(graph, pos, with_labels=False, node_size=1,arrowsize = 2,width = 0.1, node_color = 'gray')
plt.show()


In [54]:
import networkx as nx
import torch
from torch_geometric.nn import GCNConv
from torch_geometric.utils import from_networkx
import torch.nn.functional as F
from torch.nn import Linear

# 2. Feature Engineering
degree_centrality = nx.degree_centrality(graph)
in_degree_centrality = nx.in_degree_centrality(graph)
out_degree_centrality = nx.out_degree_centrality(graph)
eigenvector_centrality = nx.eigenvector_centrality(graph)
clustering_coefficient = nx.clustering(graph)

# Store features as node attributes
nx.set_node_attributes(graph, degree_centrality, 'degree')
nx.set_node_attributes(graph, in_degree_centrality, 'in_degree')
nx.set_node_attributes(graph, out_degree_centrality, 'out_degree')
nx.set_node_attributes(graph, eigenvector_centrality, 'eigenvector_degree')


# 3. Prepare features for PyTorch Geometric
features = {}  
for node in graph.nodes():
    features[node] = [
        graph.nodes[node]['degree'],
        graph.nodes[node]['in_degree'],
        graph.nodes[node]['out_degree'],
        graph.nodes[node]['degree'],

    ]

# 4. Convert to PyTorch Geometric data format
data = from_networkx(graph)
data.x = torch.tensor([features[node] for node in graph.nodes()], dtype=torch.float)

class GNNModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_class):
        super().__init__()
        self.num_classes = num_class
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
        self.classifier = Linear(2, self.num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        
        out = self.classifier(x)
        return out, x

# 6. Instantiate the model and optimizer (example)
input_dim = data.x.shape[1]  # Number of features
hidden_dim = 16  # You can adjust this
output_dim =  2
num_class = 2
model = GNNModel(input_dim, hidden_dim, output_dim, num_class)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


In [55]:
def train():
    model.train()
    optimizer.zero_grad()
    output = model(data.x, data.edge_index)
    # Calculate reconstruction loss (example - adapt to your needs)
    loss = torch.nn.MSELoss()(output, data.x)  
    loss.backward()
    optimizer.step()
    return loss.item()

numepoch = 10
for epoch in range(numepoch): 
    loss = train()
    print(f'Epoch: {epoch+1}, Loss: {loss:.4f}')

AttributeError: 'tuple' object has no attribute 'size'

In [49]:
data.num_features

4

In [56]:
from torchviz import make_dot
# Generate the graph
#  Use your actual data from NetworkX
x = data.x
edge_index = data.edge_index

# Generate the graph
output = model(x, edge_index)
graph = make_dot(output, params=dict(model.named_parameters()))

# Save the graph as an image
graph.render("gnn_model_structure", format="png") 

'gnn_model_structure.png'